In [1]:
import os
import cv2
import numpy as np
from numpy import dot
from numpy.linalg import norm
import sys
import glob
import json
import h5py
import math
from tqdm import tqdm
import torch
import torchvision
import torchvision.transforms as trn
import torchvision.models as models
import torchvision.ops.roi_align as roi_align

from c4c_modules.until_module import PreTrainedModel, AllGather, CrossEn
from c4c_modules.module_cross import CrossModel, CrossConfig, Transformer as TransformerClip

from c4c_modules.module_clip import CLIP, convert_weights
from torch.nn.utils.rnn import pad_packed_sequence, pack_padded_sequence
import pickle
import pathlib

In [2]:
from c4c_modules.file_utils import PYTORCH_PRETRAINED_BERT_CACHE
from c4c_modules.modeling import CLIP4Clip
from c4c_modules.optimization import BertAdam
from util import parallel_apply, get_logger

device = torch.device('cuda')

In [3]:
# Argument
class args:
    msvd = True # or msvd = False for MSR-VTT
    slice_framepos=2
    dset ='../' # change based on dataset location
    save_path = '../extracted_features'
    max_frames = 20
    eval_frame_order = 0 
    output_dir='pretrained'
    cache_dir=''
    
    features_path='..'
    msrvtt_csv ='msrvtt.csv'
    data_path ='MSRVTT_data.json'
    max_words=32
    feature_framerate=1
    cross_model="cross-base"
    local_rank=0

In [4]:
#MSVD 
if args.msvd:
    dset_path = os.path.join(os.path.join(args.dset,'dataset'),'MSVD')
    features_path = os.path.join(dset_path,'raw') # video .avi    
    name_list = glob.glob(features_path+os.sep+'*')
    args.features_path = features_path

    url2id = {}
    data_path = os.path.join(os.path.join(dset_path,'captions','youtube_mapping.txt'))
    args.data_path = data_path
    for line in open(data_path,'r').readlines():
        url2id[line.strip().split(' ')[0]] = line.strip().split(' ')[-1]

    path_to_saved_models = f"{args.save_path}/msvd"
    pathlib.Path(path_to_saved_models).mkdir(parents=True, exist_ok=True)
    save_file = path_to_saved_models+'/MSVD_Clip4Clip_features.pickle'
    args.max_words =30
    
    # Load video to dataloader
    %run ../dataloaders/dataloader_msvd.py import MSVD_Loader
    
    videos= MSVD_Loader(
        data_path=args.data_path,
        features_path=args.features_path,
        max_words=args.max_words,
        feature_framerate=args.feature_framerate,
        max_frames=args.max_frames,
        frame_order=args.eval_frame_order,
        slice_framepos=args.slice_framepos,
        transform_type = 0,
    ) 
#MSR-VTT    
else:
  
    dset_path = os.path.join(os.path.join(args.dset,'dataset'),'MSRVTT')
    features_path = os.path.join(dset_path,'raw') 
    args.features_path = features_path
    data_path=os.path.join(dset_path,'MSRVTT_data.json')
    args.data_path = data_path
    args.msrvtt_csv = os.path.join(dset_path,'msrvtt.csv')
    name_list = glob.glob(features_path+os.sep+'*')
    
    path_to_saved_models = "extracted/msrvtt"
    pathlib.Path(path_to_saved_models).mkdir(parents=True, exist_ok=True)
    save_file = path_to_saved_models+'/MSRVTT_Clip4Clip_features.pickle'
    args.max_words =73
    
    #Load video to dataloader
    %run ../dataloaders/dataloader_msrvtt.py import MSRVTT_RawDataLoader
    videos= MSRVTT_RawDataLoader(
        csv_path=args.msrvtt_csv,
        features_path=args.features_path,
        max_words=args.max_words,
        feature_framerate=args.feature_framerate,
        max_frames=args.max_frames,
        frame_order=args.eval_frame_order,
        slice_framepos=args.slice_framepos,
        transform_type = 0,
)

Video number: 1970
Id number: 1970


### Load the CLIP4Clip pretrained models

In [5]:
epoch = 5
model_file = os.path.join(args.output_dir, "pytorch_model.bin.{}".format(epoch-1))
model_state_dict = torch.load(model_file, map_location='cpu')
cache_dir = args.cache_dir if args.cache_dir else os.path.join(str(PYTORCH_PRETRAINED_BERT_CACHE), 'distributed')
model = CLIP4Clip.from_pretrained(args.cross_model, cache_dir=cache_dir, state_dict=model_state_dict, task_config=args)

/var/folders/5v/mbgy1mns1nj1h8c862ctlkrm0000gn/T/ipykernel_25848/4213881961.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_state_dict = torch.load(model_file, map

In [6]:
model

CLIP4Clip(
  (clip): CLIP(
    (visual): VisualTransformer(
      (conv1): Conv2d(3, 768, kernel_size=(32, 32), stride=(32, 32), bias=False)
      (ln_pre): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (transformer): Transformer(
        (resblocks): Sequential(
          (0): ResidualAttentionBlock(
            (attn): MultiheadAttention(
              (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
            )
            (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (mlp): Sequential(
              (c_fc): Linear(in_features=768, out_features=3072, bias=True)
              (gelu): QuickGELU()
              (c_proj): Linear(in_features=3072, out_features=768, bias=True)
            )
            (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          )
          (1): ResidualAttentionBlock(
            (attn): MultiheadAttention(
              (out_proj): NonDynamicallyQuantizableLi

In [7]:
clip = model.clip.to(device)

### Extract clip features

In [8]:

clip.eval()

with torch.no_grad():
    data ={}
    stop = False
    with open(save_file, 'wb') as handle:

        for i in tqdm(range(len(videos))):

            video_id,video,video_mask = videos[i]

            tensor = video[0]
            tensor = tensor[video_mask[0]==1,:]
            tensor = torch.as_tensor(tensor).float()
            video_frame,num,channel,h,w = tensor.shape
            tensor = tensor.view(video_frame*num, channel, h, w)

            video_frame,channel,h,w = tensor.shape


            output = clip.encode_image(tensor.to(device), video_frame=video_frame).float().to(device)
            output = output.detach().cpu().numpy()
            data[video_id]=output

            del output
        pickle.dump(data, handle, protocol=pickle.HIGHEST_PROTOCOL)


  0%|          | 3/1970 [00:11<2:05:18,  3.82s/it]


KeyboardInterrupt: 